In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy.stats import norm
from math import sqrt

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

# Create output folder
output_path = "../outputs/m3/"
os.makedirs(output_path, exist_ok=True)

print("Setup Complete")

Setup Complete


In [2]:
master_df = pd.read_csv("../data/processed/master_dataset.csv")

master_df['order_purchase_timestamp'] = pd.to_datetime(
    master_df['order_purchase_timestamp']
)

print("Data Loaded:", master_df.shape)
master_df.head()


Data Loaded: (109872, 30)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,9350.0,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,31570.0,belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,14840.0,guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,pet_shop,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,31842.0,belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,stationery,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,8752.0,mogi das cruzes,SP


In [3]:
product_summary = master_df.groupby('product_id').agg(
    total_units=('order_id', 'count'),
    total_revenue=('price', 'sum'),
    avg_price=('price', 'mean')
).reset_index()

master_df['date'] = master_df['order_purchase_timestamp'].dt.date

daily_demand = master_df.groupby(
    ['product_id', 'date']
).size().reset_index(name='daily_units')

demand_stats = daily_demand.groupby('product_id').agg(
    avg_daily_demand=('daily_units', 'mean'),
    std_daily_demand=('daily_units', 'std')
).reset_index()

product_master = product_summary.merge(
    demand_stats,
    on='product_id',
    how='left'
)

product_master.head()


,product_id,total_units,total_revenue,avg_price,avg_daily_demand,std_daily_demand
0,00066f42aeeb9f3007548bb9d3f33c38,1,101.65,101.65,1.0,NaN
1,00088930e925c41fd95ebfe695fd2655,1,129.90,129.90,1.0,NaN
2,0009406fd7479715e4bef61dd91f2462,1,229.00,229.00,1.0,NaN
3,000b8f95fcb9e0096488278317764d19,2,117.80,58.90,1.0,0.0
4,000d9be29b5207b54e86aa1b1ac54872,1,199.00,199.00,1.0,NaN


In [4]:
selected_products = product_master.sort_values(
    by='total_units',
    ascending=False
).head(20)

print("Selected Top 20 Products")
selected_products.head()


Selected Top 20 Products


,product_id,total_units,total_revenue,avg_price,avg_daily_demand,std_daily_demand
21528,aca2eb7d00ea1a7b8ebd4e68314663af,520,37104.30,71.354423,3.040936,2.322161
8398,422879e10f46682990de24d770e7f83d,484,26577.22,54.911612,2.469388,2.716348
19211,99a4788cb24856965c36a24e339b6058,477,42049.66,88.154423,1.813688,1.241568
7181,389d119b48cf3043d311335e499d9c6b,390,21336.79,54.709718,2.063492,1.814888
6902,368c6c730842d78016ad823897a372db,388,21056.80,54.270103,2.179775,2.183978


In [5]:
S = 100          # Ordering cost
H_rate = 0.20    # Holding rate
L = 7            # Lead time (days)
Z = 1.65         # 95% service level

inventory_df = selected_products.copy()

days_observed = (
    master_df['order_purchase_timestamp'].max() -
    master_df['order_purchase_timestamp'].min()
).days

print("Parameters Set")


Parameters Set


In [6]:
inventory_df['Annual_Demand'] = (
    inventory_df['total_units'] / days_observed
) * 365

inventory_df['Unit_Cost'] = inventory_df['avg_price']

inventory_df['Holding_Cost'] = (
    inventory_df['Unit_Cost'] * H_rate
)

inventory_df['EOQ'] = np.sqrt(
    (2 * inventory_df['Annual_Demand'] * S) /
    inventory_df['Holding_Cost']
)

inventory_df['Orders_per_Year'] = (
    inventory_df['Annual_Demand'] /
    inventory_df['EOQ']
)

inventory_df['Annual_Ordering_Cost'] = (
    inventory_df['Orders_per_Year'] * S
)

inventory_df['Annual_Holding_Cost'] = (
    (inventory_df['EOQ'] / 2) *
    inventory_df['Holding_Cost']
)

inventory_df['Total_Inventory_Cost'] = (
    inventory_df['Annual_Ordering_Cost'] +
    inventory_df['Annual_Holding_Cost']
)

inventory_df.head()


,product_id,total_units,total_revenue,avg_price,avg_daily_demand,std_daily_demand,Annual_Demand,Unit_Cost,Holding_Cost,EOQ,Orders_per_Year,Annual_Ordering_Cost,Annual_Holding_Cost,Total_Inventory_Cost
21528,aca2eb7d00ea1a7b8ebd4e68314663af,520,37104.30,71.354423,3.040936,2.322161,315.806988,71.354423,14.270885,66.527380,4.747023,474.702280,474.702280,949.404560
8398,422879e10f46682990de24d770e7f83d,484,26577.22,54.911612,2.469388,2.716348,293.943428,54.911612,10.982322,73.164394,4.017575,401.757481,401.757481,803.514961
19211,99a4788cb24856965c36a24e339b6058,477,42049.66,88.154423,1.813688,1.241568,289.692180,88.154423,17.630885,57.325299,5.053479,505.347871,505.347871,1010.695742
7181,389d119b48cf3043d311335e499d9c6b,390,21336.79,54.709718,2.063492,1.814888,236.855241,54.709718,10.941944,65.797480,3.599762,359.976158,359.976158,719.952316
6902,368c6c730842d78016ad823897a372db,388,21056.80,54.270103,2.179775,2.183978,235.640599,54.270103,10.854021,65.893828,3.576065,357.606482,357.606482,715.212964


In [7]:
inventory_df['Safety_Stock'] = (
    Z *
    inventory_df['std_daily_demand'] *
    np.sqrt(L)
)

inventory_df['ROP'] = (
    (inventory_df['avg_daily_demand'] * L) +
    inventory_df['Safety_Stock']
)

inventory_df['Max_Inventory'] = (
    inventory_df['ROP'] +
    inventory_df['EOQ']
)

inventory_df['Avg_Inventory'] = (
    (inventory_df['EOQ'] / 2) +
    inventory_df['Safety_Stock']
)

inventory_df.head()


,product_id,total_units,total_revenue,avg_price,avg_daily_demand,std_daily_demand,Annual_Demand,Unit_Cost,Holding_Cost,EOQ,Orders_per_Year,Annual_Ordering_Cost,Annual_Holding_Cost,Total_Inventory_Cost,Safety_Stock,ROP,Max_Inventory,Avg_Inventory
21528,aca2eb7d00ea1a7b8ebd4e68314663af,520,37104.30,71.354423,3.040936,2.322161,315.806988,71.354423,14.270885,66.527380,4.747023,474.702280,474.702280,949.404560,10.137370,31.423920,97.951300,43.401060
8398,422879e10f46682990de24d770e7f83d,484,26577.22,54.911612,2.469388,2.716348,293.943428,54.911612,10.982322,73.164394,4.017575,401.757481,401.757481,803.514961,11.858188,29.143902,102.308297,48.440385
19211,99a4788cb24856965c36a24e339b6058,477,42049.66,88.154423,1.813688,1.241568,289.692180,88.154423,17.630885,57.325299,5.053479,505.347871,505.347871,1010.695742,5.420053,18.115870,75.441169,34.082702
7181,389d119b48cf3043d311335e499d9c6b,390,21336.79,54.709718,2.063492,1.814888,236.855241,54.709718,10.941944,65.797480,3.599762,359.976158,359.976158,719.952316,7.922876,22.367321,88.164801,40.821616
6902,368c6c730842d78016ad823897a372db,388,21056.80,54.270103,2.179775,2.183978,235.640599,54.270103,10.854021,65.893828,3.576065,357.606482,357.606482,715.212964,9.534132,24.792559,90.686386,42.481045


In [8]:
current_order_qty = 200

inventory_df['Current_Cost'] = (
    (inventory_df['Annual_Demand'] / current_order_qty) * S +
    (current_order_qty / 2) *
    inventory_df['Holding_Cost']
)

inventory_df['Proposed_Cost'] = (
    (inventory_df['Annual_Demand'] / inventory_df['EOQ']) * S +
    (inventory_df['EOQ'] / 2 + inventory_df['Safety_Stock']) *
    inventory_df['Holding_Cost']
)

inventory_df['Savings'] = (
    inventory_df['Current_Cost'] -
    inventory_df['Proposed_Cost']
)

inventory_df['Savings_%'] = (
    inventory_df['Savings'] /
    inventory_df['Current_Cost']
) * 100

inventory_df.head()


,product_id,total_units,total_revenue,avg_price,avg_daily_demand,std_daily_demand,Annual_Demand,Unit_Cost,Holding_Cost,EOQ,Orders_per_Year,Annual_Ordering_Cost,Annual_Holding_Cost,Total_Inventory_Cost,Safety_Stock,ROP,Max_Inventory,Avg_Inventory,Current_Cost,Proposed_Cost,Savings,Savings_%
21528,aca2eb7d00ea1a7b8ebd4e68314663af,520,37104.30,71.354423,3.040936,2.322161,315.806988,71.354423,14.270885,66.527380,4.747023,474.702280,474.702280,949.404560,10.137370,31.423920,97.951300,43.401060,1584.991956,1094.073800,490.918156,30.972912
8398,422879e10f46682990de24d770e7f83d,484,26577.22,54.911612,2.469388,2.716348,293.943428,54.911612,10.982322,73.164394,4.017575,401.757481,401.757481,803.514961,11.858188,29.143902,102.308297,48.440385,1245.203945,933.745406,311.458540,25.012653
19211,99a4788cb24856965c36a24e339b6058,477,42049.66,88.154423,1.813688,1.241568,289.692180,88.154423,17.630885,57.325299,5.053479,505.347871,505.347871,1010.695742,5.420053,18.115870,75.441169,34.082702,1907.934559,1106.256065,801.678494,42.018134
7181,389d119b48cf3043d311335e499d9c6b,390,21336.79,54.709718,2.063492,1.814888,236.855241,54.709718,10.941944,65.797480,3.599762,359.976158,359.976158,719.952316,7.922876,22.367321,88.164801,40.821616,1212.621980,806.643981,405.977998,33.479353
6902,368c6c730842d78016ad823897a372db,388,21056.80,54.270103,2.179775,2.183978,235.640599,54.270103,10.854021,65.893828,3.576065,357.606482,357.606482,715.212964,9.534132,24.792559,90.686386,42.481045,1203.222361,818.696625,384.525736,31.957994


In [9]:
inventory_df['Mean_Lead_Time_Demand'] = (
    inventory_df['avg_daily_demand'] * L
)

inventory_df['Z_score_current'] = (
    (current_order_qty -
     inventory_df['Mean_Lead_Time_Demand']) /
    (inventory_df['std_daily_demand'] * np.sqrt(L))
)

inventory_df['Stockout_Risk_Current'] = (
    1 - norm.cdf(inventory_df['Z_score_current'])
)

inventory_df['Z_score_new'] = (
    (inventory_df['ROP'] -
     inventory_df['Mean_Lead_Time_Demand']) /
    (inventory_df['std_daily_demand'] * np.sqrt(L))
)

inventory_df['Stockout_Risk_New'] = (
    1 - norm.cdf(inventory_df['Z_score_new'])
)

inventory_df.head()


,product_id,total_units,total_revenue,avg_price,avg_daily_demand,std_daily_demand,Annual_Demand,Unit_Cost,Holding_Cost,EOQ,Orders_per_Year,Annual_Ordering_Cost,Annual_Holding_Cost,Total_Inventory_Cost,Safety_Stock,ROP,Max_Inventory,Avg_Inventory,Current_Cost,Proposed_Cost,Savings,Savings_%,Mean_Lead_Time_Demand,Z_score_current,Stockout_Risk_Current,Z_score_new,Stockout_Risk_New
21528,aca2eb7d00ea1a7b8ebd4e68314663af,520,37104.30,71.354423,3.040936,2.322161,315.806988,71.354423,14.270885,66.527380,4.747023,474.702280,474.702280,949.404560,10.137370,31.423920,97.951300,43.401060,1584.991956,1094.073800,490.918156,30.972912,21.286550,29.088135,0.0,1.65,0.049471
8398,422879e10f46682990de24d770e7f83d,484,26577.22,54.911612,2.469388,2.716348,293.943428,54.911612,10.982322,73.164394,4.017575,401.757481,401.757481,803.514961,11.858188,29.143902,102.308297,48.440385,1245.203945,933.745406,311.458540,25.012653,17.285714,25.423662,0.0,1.65,0.049471
19211,99a4788cb24856965c36a24e339b6058,477,42049.66,88.154423,1.813688,1.241568,289.692180,88.154423,17.630885,57.325299,5.053479,505.347871,505.347871,1010.695742,5.420053,18.115870,75.441169,34.082702,1907.934559,1106.256065,801.678494,42.018134,12.695817,57.020092,0.0,1.65,0.049471
7181,389d119b48cf3043d311335e499d9c6b,390,21336.79,54.709718,2.063492,1.814888,236.855241,54.709718,10.941944,65.797480,3.599762,359.976158,359.976158,719.952316,7.922876,22.367321,88.164801,40.821616,1212.621980,806.643981,405.977998,33.479353,14.444444,38.643374,0.0,1.65,0.049471
6902,368c6c730842d78016ad823897a372db,388,21056.80,54.270103,2.179775,2.183978,235.640599,54.270103,10.854021,65.893828,3.576065,357.606482,357.606482,715.212964,9.534132,24.792559,90.686386,42.481045,1203.222361,818.696625,384.525736,31.957994,15.258427,31.971826,0.0,1.65,0.049471


In [10]:
abc_df = product_summary.copy()

total_revenue = abc_df['total_revenue'].sum()

abc_df = abc_df.sort_values(
    by='total_revenue',
    ascending=False
).reset_index(drop=True)

abc_df['Revenue_%'] = (
    abc_df['total_revenue'] / total_revenue
) * 100

abc_df['Cumulative_%'] = abc_df['Revenue_%'].cumsum()

conditions = [
    abc_df['Cumulative_%'] <= 80,
    (abc_df['Cumulative_%'] > 80) &
    (abc_df['Cumulative_%'] <= 95),
    abc_df['Cumulative_%'] > 95
]

choices = ['A', 'B', 'C']

abc_df['ABC_Class'] = np.select(
    conditions,
    choices,
    default='C'
)

abc_df.head()


,product_id,total_units,total_revenue,avg_price,Revenue_%,Cumulative_%,ABC_Class
0,bb50f2e236e5eea0100680137654686c,194,63560.00,327.628866,0.482254,0.482254,A
1,6cdd53843498f92890544667809f1595,153,53652.30,350.668627,0.407080,0.889334,A
2,d6160fb7873f184099d9bc95e30376af,33,45949.35,1392.404545,0.348635,1.237970,A
3,d1c427060a0f73f6b889a5c7c61f2ac4,332,45620.56,137.411325,0.346141,1.584110,A
4,99a4788cb24856965c36a24e339b6058,477,42049.66,88.154423,0.319047,1.903157,A


In [11]:
total_savings = inventory_df['Savings'].sum()
total_investment = (
    inventory_df['Safety_Stock'] *
    inventory_df['Unit_Cost']
).sum()

ROI = (total_savings / total_investment) * 100

print("Total Savings:", total_savings)
print("Total Investment:", total_investment)
print("ROI %:", ROI)


Total Savings: 22888.353096296873
Total Investment: 11873.626105562596
ROI %: 192.7663284392462


In [12]:
inventory_df.to_csv(
    output_path + "inventory_optimization_results.csv",
    index=False
)

abc_df.to_csv(
    output_path + "abc_classification_results.csv",
    index=False
)

abc_summary = abc_df.groupby(
    'ABC_Class'
)['Revenue_%'].sum().reset_index()

abc_summary.to_csv(
    output_path + "abc_summary_percentage.csv",
    index=False
)

summary_metrics = pd.DataFrame({
    "Total_Savings": [total_savings],
    "Total_Investment": [total_investment],
    "ROI_%": [ROI]
})

summary_metrics.to_csv(
    output_path + "executive_summary_metrics.csv",
    index=False
)

print("All M3 Outputs Saved Successfully")


All M3 Outputs Saved Successfully


In [13]:
# ABC Pareto
plt.figure(figsize=(10,6))
plt.plot(abc_df['Cumulative_%'])
plt.axhline(80, linestyle='--')
plt.axhline(95, linestyle='--')
plt.title("ABC Pareto Curve")
plt.savefig(output_path + "abc_pareto_chart.png")
plt.close()

# EOQ vs Savings
plt.figure(figsize=(10,6))
plt.scatter(inventory_df['EOQ'],
            inventory_df['Savings'])
plt.title("EOQ vs Savings")
plt.savefig(output_path + "eoq_vs_savings.png")
plt.close()

# Stockout Risk Comparison
plt.figure(figsize=(6,6))
plt.bar(
    ['Current', 'Proposed'],
    [
        inventory_df['Stockout_Risk_Current'].mean(),
        inventory_df['Stockout_Risk_New'].mean()
    ]
)
plt.title("Stockout Risk Comparison")
plt.savefig(output_path + "stockout_risk_comparison.png")
plt.close()

print("Charts Saved")


Charts Saved
